In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm
!python -m spacy download en_core_web_trf
!python -m spacy download ru_core_news_lg

!pip install beautifulsoup4
!pip install google
!pip install html2text

# [+] Сделать запрос к поисковику, найти штук 10-20 ссылок

Собрать несколько текстов

https://www.geeksforgeeks.org/performing-google-search-using-python-code/

In [1]:
def search_google(search_string : str) -> (list, str):
    try:
        import googlesearch
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")
        return ([], "Не установлен python-модуль google")
    result = []
    try:
        result = googlesearch.search(search_string, tld="co.in", num=20, stop=20, pause=2)
    except:
        return ([], "Поиск в Google не удался, возможно, превышен лимит, попробуйте сменить IP или повторить позже")
    return (list(result), None)

search_string = "Эльбрус 8С"
search_result, err_message = search_google(search_string)
# for j in search_result:
#     print(j)
print(search_result)

['https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-8%D0%A1', 'https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81_(%D0%BF%D1%80%D0%BE%D1%86%D0%B5%D1%81%D1%81%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D0%B0%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D0%B0)', 'https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-2%D0%A13', 'https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-4%D0%A1', 'https://ru.wikipedia.org/wiki/%D0%9C%D0%A6%D0%A1%D0%A2', 'http://www.mcst.ru/elbrus-8c', 'https://en.wikipedia.org/wiki/Elbrus-8S', 'http://www.mcst.ru/elbrus-8c0', 'https://habr.com/ru/post/647277/', 'https://www.tadviser.ru/index.php/%D0%9F%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82:%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-8%D0%A1', 'https://www.ixbt.com/news/2021/05/30/s-t-a-l-k-e-r-8.html', 'https://3dnews.ru/1009369', 'https://www.kommersant.ru/doc/5192750', 'https://icl-techno.ru/catalog/desktop/monoblok-saferay-basicray

# [+] Получить страницы по ссылкам и выделить контексты

In [16]:
import re
import requests
import bs4
from bs4 import BeautifulSoup
import webbrowser
import html2text

def get_static_page_text(url : str) -> str:
    """
    Запрос к URL и выделение текста через requests.get().text
    """
    request_result=requests.get( url )
    return str(request_result.text)

def get_static_page_text2(url : str) -> str:
    """
    Запрос к URL и выделение текста через html2text
    """
    request_result=requests.get( url )
    # raw=html2text.HTML2Text().handle(request_result.text)
    # raw=html2text.html2text(request_result.text)
    
    to_text = html2text.HTML2Text()
    to_text.ignore_links = False
    extracted_text = to_text.handle(request_result.text)
    
    # raw=html2text.html2text(request_result.content)
    # utf8_text=raw.encode('utf-8') # если нужен вывод в bytes
    return extracted_text # utf8_text если нужен вывод в bytes

def get_static_page_text3(url : str) -> str:
    """
    Запрос к URL и выделение текста через BeautifulSoup
    """
    pass

def get_word_contexts(
    source_text : str,
    target_text : str, # строка, для которой нужно выделить контекст
    words_before : int, # число слов перед найденным target_text, которые включаются в контекст
    words_after : int # число слов после найденного target_text, которые включаются в контекст
) -> list:
    """
    Выделение из текста source_text набора фрагментов текста (т.е. конеткстов в формате list(str)),
    включающих слово target_text, а также words_before слов перед словом target_text
    и words_after слов после слова target_text
    """
    # знаки препинания [\W&&\S]
    # слова с кириллицей [\wа-яА-Я]
    # regex_to_find_context = rf"(?:[\w]+\W+)\{{0,{words_before}}}{target_text}(?:\W+[\w]+)\{{0,{words_after}}}"
    regex_to_find_context = rf"(?:[\wа-яА-Я]+[^\wа-яА-Я]+){{0,{words_before}}}{target_text}(?:[^\wа-яА-Я]+[\wа-яА-Я]+){{0,{words_after}}}"
    # list_of_contexts = []
    # source_lower = source_text.lower()
    # target_lower = target_text.lower()
    # source_lower.find(target_lower)
    list_of_contexts = re.findall(regex_to_find_context, source_text, re.IGNORECASE)
    return list_of_contexts

# print(requests.get( search_result[0] ).text)

text = ""
for url in search_result[0:1]:
    page_text=get_static_page_text2(url)
    word_contexts = get_word_contexts(source_text= page_text,
                                    target_text = "Эльбрус-8С",
                                    words_before = 5,
                                    words_after = 5
                                   )
    
    # soup = BeautifulSoup(html, features="html.parser")
    # text = text = ' '.join(soup.stripped_strings)
    print("Исходный текст:\n\n", page_text,"\n\n")
    print("Найденные совпадения:\n\n", word_contexts,"\n\n")
    print("Число совпадений:\n\n", len(word_contexts),"\n\n")

Исходный текст:

 # Эльбрус-8С

Материал из Википедии — свободной энциклопедии

Текущая версия страницы пока [не
проверялась](/wiki/%D0%92%D0%B8%D0%BA%D0%B8%D0%BF%D0%B5%D0%B4%D0%B8%D1%8F:%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_%D1%81%D1%82%D0%B0%D1%82%D0%B5%D0%B9/%D0%9F%D0%BE%D1%8F%D1%81%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B4%D0%BB%D1%8F_%D1%87%D0%B8%D1%82%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9
"Википедия:Проверка статей/Пояснение для читателей") опытными участниками и
может значительно отличаться от
[версии](https://ru.wikipedia.org/w/index.php?title=%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-8%D0%A1&stable=1),
проверенной 4 марта 2022 года; проверки требуют [17
правок](https://ru.wikipedia.org/w/index.php?title=%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-8%D0%A1&oldid=120414995&diff=cur&diffonly=0).

Текущая версия страницы пока [не
проверялась](/wiki/%D0%92%D0%B8%D0%BA%D0%B8%D0%BF%D0%B5%D0%B4%D0%B8%D1%8F:%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_%D1%81%D1%82%D0%B0%D1%82%D0

# Провести классификацию (найти категорию товара из списка категорий)

https://towardsdatascience.com/how-to-build-a-fast-most-similar-words-method-in-spacy-32ed104fe498

https://towardsdatascience.com/clustering-product-names-with-python-part-1-f9418f8705c8



In [7]:
import pandas as pd

# список категорий
df = pd.read_csv("База_продукты_Кирилл.csv", header=None, names=range(3))
table_names = ["Inventory", "HP BladeSystem Rack", "Network Interface"]
groups = df[0].isin(table_names).cumsum()
tables = {g.iloc[0,0]: g.iloc[1:] for k,g in df.groupby(groups)}

#TODO классификация

# [+] Выделить организации

* named entity recognition (ner),
* прямой поиск из базы названий предприятий
* эластичный поиск

https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7dahttps://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da



In [13]:
import spacy

# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("ru_core_news_sm")
# nlp = spacy.load("ru_core_news_lg")

# text = "Эльбрус-8С и Эльбрус-8СВ — 8-ядерные процессоры с архитектурой «Эльбрус» для персональных компьютеров и серверов. Разработаны российской компанией МЦСТ."
# text = "АО «НПП «Пульсар» входит в состав холдинга «Росэлектроника», управляющей компанией которого является Объединенная приборостроительная корпорация."
# text = "Apple is looking at buying U.K. startup for $1 billion"
# print("Исходный текст:\t", text,"\n")
doc = nlp(text)

# for token in doc:
#     # print(token)
#     # print(token.text, token.tag_)
#     # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#     #         token.shape_, token.is_alpha, token.is_stop)
orgs = []
stopwords = ["Википедия", "Википедии", "Коммерсантъ"]
for url in search_result:
    try:
        page_text=get_static_page_text2(url)
        
        doc = nlp(page_text)
        orgs_for_one_site = []
        print("url = \t", url, "\n")
        for ent in doc.ents:
            if (ent.label_ == "ORG") and (ent.text not in stopwords):
                print(ent.text, "\t", ent.label_)
                orgs_for_one_site.append(ent.text)
        print("\n")
        orgs_for_one_site = list(dict.fromkeys(orgs_for_one_site))
        orgs.extend(orgs_for_one_site)
    except:
        continue
    
orgs.sort(key= lambda x:x[0])
print(orgs)
# https://spacy.io/usage/linguistic-features#named-entities
# print("Выделение категорий")
# for ent in doc.ents:
#     print(ent.text, "\t", ent.label_)

url = 	 https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D1%8C%D0%B1%D1%80%D1%83%D1%81-8%D0%A1 

МГц 	 ORG
МЦСТ](/wiki/%D0%9C%D0%A6%D0%A1%D0%A2
 	 ORG
Синьчжу 	 ORG
Тайвань 	 ORG
TSMC 	 ORG
Эльбрус 	 ORG
Российского правительства по
[импортозамещению](/wiki/%D0%98%D0%BC%D0%BF%D0%BE%D1%80%D1%82%D0%BE%D0%B7%D0%B0%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8
 	 ORG
ALT Linux 	 ORG
ЛИНТЕР 	 ORG
Эльбрус-8С 	 ORG
Института
электронных управляющих машин ([ИНЭУМ](/wiki/%D0%98%D0%9D%D0%AD%D0%A3%D0%9C
"ИНЭУМ")) 	 ORG
МЦСТ](/wiki/%D0%9C%D0%A6%D0%A1%D0%A2 "МЦСТ")
 	 ORG
ИНЭУМ 	 ORG
ИНЭУМ 	 ORG
Системы управления 	 ORG
Объединённую приборостроительную
корпорацию](/wiki/%D0%9E%D0%B1%D1%8A%D0%B5%D0%B4%D0%B8%D0%BD%D1%91%D0%BD%D0%BD%D0%B0%D1%8F_%D0%BF%D1%80%D0%B8%D0%B1%D0%BE%D1%80%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D0%BA%D0%BE%D1%80%D0%BF%D0%BE%D1%80%D0%B0%D1%86%D0%B8%D1%8F
 	 ORG
Объединённая приборостроительная корпорация"

# По частоте выделить возможного производителя